In [89]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [91]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [92]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "5280728100755326513", 2, "feature")

In [93]:
emptyStruct = Lar.Struct()

LinearAlgebraicRepresentation.Struct(Any[], Any, "6837444746729184950", Any, "feature")

## Vecchia funzione

In [111]:
function embedTraversal(cloned::Lar.Struct,obj::Lar.Struct,n::Int,suffix::String)
	for i=1:length(obj.body)
		if isa(obj.body[i],Matrix)
			mat = obj.body[i]
			d,d = size(mat)
			newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
			for h in range(1, length=d)
				for k in range(1, length=d)
					newMat[h,k]=mat[h,k]
				end
			end
			push!(cloned.body,[newMat])
		elseif (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==3
			V,FV,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,FV,EV)])
		elseif  (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==2
			V,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,EV)])
		elseif isa(obj.body[i],Lar.Struct)
			newObj = Struct()
			newObj.box = [ [obj.body[i].box[1];zeros(dimadd)],
							[obj.body[i].box[2];zeros(dimadd)] ]
			newObj.category = (obj.body[i]).category
			push!(cloned.body,[embedTraversal(newObj,obj.body[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversal (generic function with 2 methods)

In [110]:
@btime embedTraversal(emptyStruct,x,1,"New")

  2.775 μs (20 allocations: 1.53 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]  …  [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 

## Nuove funzioni

In [96]:
function embedTraversalMatrix(objBody)
	mat = objBody[i]
	d,d = size(mat)
	newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
	for h in range(1, length=d)
		for k in range(1, length=d)
			newMat[h,k]=mat[h,k]
		end
	end
	return newMat
end

embedTraversalMatrix (generic function with 1 method)

In [97]:
function embedTraversalTupleOrArray(n,V)
	dimadd = n
	ncols = size(V,2)
	nmat = zeros(dimadd,ncols)
	V = [V;nmat]
	return V
end

embedTraversalTupleOrArray (generic function with 1 method)

In [98]:
function embedTraversalStruct(objBody)
	newObj = Struct()
	newObj.box = [ [objBody[i].box[1];zeros(dimadd)],
				   [objBody[i].box[2];zeros(dimadd)] ]
	newObj.category = (objBody[i]).category
	return
end

embedTraversalStruct (generic function with 1 method)

In [99]:
function embedTraversal(cloned,obj,n,suffix)
	objBody = obj.body
	for i=1:length(objBody)
		if isa(objBody[i],Matrix)
			newMat = embedTraversalMatrix(objBody)
			push!(cloned.body,[newMat])
		elseif (isa(objBody[i],Tuple) || isa(objBody[i],Array))
			if (length(objBody[i])==3)
				V,FV,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,FV,EV)])
			elseif (length(objBody[i])==2)
				V,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,EV)])
			end
		elseif isa(objBody[i],Struct)
			newObj = embedTraversalStruct(objBody)
			push!(cloned.body,[embedTraversal(newObj,objBody[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversa (generic function with 4 methods)

In [100]:
@btime embedTraversal(emptyStruct,x,1,"New")

  2.544 μs (18 allocations: 1.39 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]  …  [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 